In [27]:
import numpy as np
import pybullet as p
import pybullet_data

In [28]:
ClientId = p.connect(p.GUI)

In [29]:
p.setAdditionalSearchPath(pybullet_data.getDataPath())

p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)                        # Enable/Disable a default GUI
p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS, 1)                    # Enable/Disable a light effect
p.configureDebugVisualizer(p.COV_ENABLE_RGB_BUFFER_PREVIEW, 0)         # Show/Hide a RGB buffer preview window
p.configureDebugVisualizer(p.COV_ENABLE_DEPTH_BUFFER_PREVIEW, 0)       # Show/Hide a Depth buffer preview window
p.configureDebugVisualizer(p.COV_ENABLE_SEGMENTATION_MARK_PREVIEW, 0)  # Show/Hide a Seg-map buffer preview window

In [30]:
PlaneId = p.loadURDF("plane.urdf")

In [31]:
p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=45, cameraPitch=-30, cameraTargetPosition=[0, 0, 0.5])

In [32]:
p.setGravity(0, 0, -9.81) # set a gravity vector
p.setTimeStep(1./240)     # set a simulation time step (240 Hz)

In [33]:
p.setRealTimeSimulation(1) # start real-time physics simulation

In [34]:
def visualizeEEframe(robotId, n=6):
    LineID = []
    for i in range(3):
        id = p.addUserDebugLine(lineFromXYZ=[0, 0, 0.06], lineToXYZ=0.1*np.eye(3)[i] + [0, 0, 0.06], lineColorRGB=np.eye(3)[i], lineWidth=5, 
                                parentObjectUniqueId=robotId, parentLinkIndex=n-1, physicsClientId=ClientId)
        LineID.append(id)
    return LineID

In [35]:
urdf_path = "./indy7_v2/model.urdf"
base_pos = [0, 0, 0]
base_quat = p.getQuaternionFromEuler([0, 0, 0])

flags = p.URDF_USE_INERTIA_FROM_FILE + p.URDF_USE_SELF_COLLISION + p.URDF_USE_SELF_COLLISION_EXCLUDE_PARENT

robotId = p.loadURDF(urdf_path, 
                     basePosition=base_pos, 
                     baseOrientation=base_quat,
                     flags=flags, 
                     physicsClientId=ClientId)
numJoints = p.getNumJoints(robotId, physicsClientId=ClientId)
jointTypes = [p.getJointInfo(robotId, i)[2] for i in range(numJoints)]
MovableJointIdx = [i for i, jointType in enumerate(jointTypes) if jointType==p.JOINT_REVOLUTE or jointType==p.JOINT_PRISMATIC]
LineID = visualizeEEframe(robotId, len(MovableJointIdx))

In [36]:
# TODO: Define functions for FK by DH
def updateDHparams(dh_parameters, joint_variables, jointTypes):
    for i, (joint_variable, jointType) in enumerate(zip(joint_variables, jointTypes)):
        if jointType == p.JOINT_FIXED:
            continue
        elif jointType == p.JOINT_REVOLUTE:
            dh_parameters[i+1][1] += joint_variable
        elif jointType == p.JOINT_PRISMATIC:
            dh_parameters[i+1][0] += joint_variable
        else:
            continue
    return dh_parameters

def getRelativeTransformation(dh_parameter):
    d, theta, r, alpha = dh_parameter
    T = [[np.cos(theta), -np.sin(theta) * np.cos(alpha), np.sin(theta) * np.sin(alpha), r * np.cos(theta)],
         [np.sin(theta), np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), r * np.sin(theta)],
         [0 , np.sin(alpha), np.cos(alpha), d],
         [0, 0, 0, 1]]
    return np.array(T)

def FK_DH(dh_parameters):
    T = np.eye(4)
    for i in dh_parameters:
        T = np.matmul(T, getRelativeTransformation(i))
    return T

# TODO: Define function to get Transformation matrix of end effector frame (tcp)
def getTfromRp(R, pos):
    T = np.eye(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = pos
    return T

def getEETransformation(robotId, n=7):
    linkState = p.getLinkState(robotId, n-1)
    pos = list(linkState[4])
    # pos[2] += 0.06
    R = p.getMatrixFromQuaternion(linkState[5])
    R = np.array(R).reshape(3,3)
    return getTfromRp(R, pos)
    # phi, theta, psi = p.getEulerFromQuaternion(linkState[5]) # XYZ Euler angle
    # cphi = np.cos(phi)
    # sphi = np.sin(phi)
    # ctheta = np.cos(theta)
    # stheta = np.sin(theta)
    # cpsi = np.cos(psi)
    # spsi = np.sin(psi)
    # R = [[ctheta * cpsi, -ctheta * spsi, stheta],
    #      [sphi * stheta * cpsi + cphi * spsi, -sphi * stheta * spsi + cphi * cpsi, -sphi * ctheta],
    #      [-cphi * stheta * cpsi + sphi * spsi, cphi * stheta * spsi + sphi * cpsi, cphi * ctheta]]
    


In [37]:
# TODO: Set arbitrary joint variables
q = np.array([0,0,0,0,0,0]) * np.pi / 180
#q = np.array([0,0,0,0,0,10]) * np.pi / 180
q = np.array([30,30,10,20,60,40]) * np.pi / 180

# TODO: Define Indy7 v2 DH parameters
PI = np.pi
dh_parameters = [[0.0775, 0, 0, 0],
                 [0.222, 0, 0, PI/2],
                 [0.109-0.0305, PI/2, 0.45, 0],
                 [-0.075, -PI/2, 0, -PI/2],
                 [0.267+0.083, 0, 0, PI/2],
                 [0.114+0.085, 0, 0, -PI/2],
                 [0.112, 0, 0, 0]] # d, theta, r, alpha

# TODO: Get EE's Transformation matrix using FK by DH
dh_parameters = updateDHparams(dh_parameters, q, jointTypes)
T_dh = FK_DH(dh_parameters)
R = T_dh[0:3, 0:3]
pos = T_dh[0:3, 3].T
p.addUserDebugLine(lineFromXYZ=pos, lineToXYZ=pos+np.matmul(R,[0.3, 0, 0]), lineColorRGB=[1, 0, 0],
                    lineWidth=5, physicsClientId=ClientId)
p.addUserDebugLine(lineFromXYZ=pos, lineToXYZ=pos+np.matmul(R,[0, 0.3, 0]), lineColorRGB=[0, 1, 0],
                    lineWidth=5, physicsClientId=ClientId)
p.addUserDebugLine(lineFromXYZ=pos, lineToXYZ=pos+np.matmul(R,[0, 0, 0.3]), lineColorRGB=[0, 0, 1],
                    lineWidth=5, physicsClientId=ClientId)

# TODO: Apply joint variables to robot and get EE's Transformation matrix
p.setJointMotorControlArray(robotId, 
                            jointIndices=MovableJointIdx, 
                            controlMode=p.POSITION_CONTROL,
                            targetPositions=q,
                            physicsClientId=ClientId)

In [38]:
from time import sleep
sleep(0.5)
T_simul = getEETransformation(robotId, numJoints)

print(np.matmul(np.linalg.inv(T_dh), T_simul)) # Error


[[ 1.00000000e+00  5.14182886e-06 -2.09207639e-05 -2.07499794e-06]
 [-5.14124972e-06  1.00000000e+00  2.76826771e-05  8.46774389e-06]
 [ 2.09209062e-05 -2.76825695e-05  9.99999999e-01  1.02054850e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [39]:
p.disconnect()